In [ ]:
"""
Goal of this notebook is to take our x-ray png files and build a convolutional neural network to predict which images are part of 'findings' and which are part of 'no-findings'

Procedure and Outline

Loading the data
1. Use ImageDataGenerator to create augmentations of the data
2. Split into training and validation subset

Training The Model
1. Load the pretrained models
2. Set trainable to False
3. Create a custom layer on top of this model -> Convolutional Layer, MaxPooling2D, Dense(128), Dense(1).
4. * Experiment with dropout to see if it helps
5. Compile the model -> Set the learning rate (tf.keras.?.adam), loss='binary_crossentropy' since it is a binary classification problem, metric=['accuracy'] (or MatthewCorrelationCoefficient see step 1 of Testing)
6. Create history = model.fit(train=(X_train, y_train), validation=(X_val, y_val), epochs=5, verbose=1, batch_size=32)

Testing
1. Create a function to get the metric (using Matthew Correlation Coefficient) -> may have to do this prior to compiling the model and set this function as the metric. Look more into this, test both.
2. Create an array, y_test = model.predict(X_test)
3. Submit and see score

Assessment
Try to find weaknesses and see where you can Improve
"""

"\nGoal of this notebook is to take our x-ray png files and build a convolutional neural network to predict which images are part of 'findings' and which are part of 'no-findings'\n\nProcedure and Outline\n\nLoading the data\n1. Use ImageDataGenerator to create augmentations of the data \n2. Split into training and validation subset\n\nTraining The Model\n1. Load the pretrained models\n2. Set trainable to False\n3. Create a custom layer on top of this model -> Convolutional Layer, MaxPooling2D, Dense(128), Dense(1). \n4. * Experiment with dropout to see if it helps\n5. Compile the model -> Set the learning rate (tf.keras.?.adam), loss='binary_crossentropy' since it is a binary classification problem, metric=['accuracy'] (or MatthewCorrelationCoefficient see step 1 of Testing)\n6. Create history = model.fit(train=(X_train, y_train), validation=(X_val, y_val), epochs=5, verbose=1, batch_size=32)\n\nTesting\n1. Create a function to get the metric (using Matthew Correlation Coefficient) ->

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# import required libraries
import numpy as np
import pandas as pd
import os

# image processing
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# machine learning libraries
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.applications import DenseNet169
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Input, BatchNormalization, Concatenate, Flatten
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau


In [5]:
train_file_path = '/content/drive/MyDrive/xray_project/Train_PNG'

datagen = ImageDataGenerator(
    rescale = 1.0/255.00,
    rotation_range = 10,
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    zoom_range = 0.1,
    validation_split = 0.2
)

train_generator = datagen.flow_from_directory(
    directory = train_file_path,
    target_size = (512, 512),
    color_mode = 'rgb',
    class_mode = 'binary',
    batch_size = 32,
    shuffle = True,
    subset = 'training'
)

validation_generator = datagen.flow_from_directory(
    directory = train_file_path,
    target_size = (512, 512),
    color_mode = 'rgb',
    class_mode = 'binary',
    batch_size = 32,
    shuffle = False,
    subset = 'validation'
)

Found 8001 images belonging to 2 classes.
Found 1999 images belonging to 2 classes.


In [6]:
input_layer = Input(shape=(512, 512, 3))

mobilenet_base = MobileNetV2(weights='imagenet', input_shape=(512, 512, 3), include_top=False)
densenet_base = DenseNet169(weights='imagenet', input_shape=(512, 512, 3), include_top=False)

# set the layers not trainable to preserve weights of pretrained model
for layer in mobilenet_base.layers:
    layer.trainable=False
for layer in densenet_base.layers:
    layer.trainable=False

model_mobilenet = mobilenet_base(input_layer)
model_mobilenet = GlobalAveragePooling2D()(model_mobilenet)
output_mobilenet = Flatten()(model_mobilenet)

model_densenet = densenet_base(input_layer)
model_densenet = GlobalAveragePooling2D()(model_densenet)
output_densenet = Flatten()(model_densenet)

merged = Concatenate()([output_mobilenet, output_densenet])

x = BatchNormalization()(merged)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(1, activation='sigmoid')(x)

model = Model(inputs=input_layer, outputs=x)

<ipython-input-6-8a6a3cdfc89b>:3: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  mobilenet_base = MobileNetV2(weights='imagenet', input_shape=(512, 512, 3), include_top=False)


In [7]:
for layer in mobilenet_base.layers[-10:]:
    layer.trainable = True

for layer in densenet_base.layers[-10:]:
    layer.trainable = True


model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

early_stopping = EarlyStopping(patience=5, restore_best_weights=True)

lr_reducer = ReduceLROnPlateau(
    monitor='val_loss',
    patience=3,
    verbose=1,
    factor=0.5,
    min_lr=0.0001
)

checkpoint_callback = ModelCheckpoint(
    filepath='model_checkpoint.weights.h5',
    save_weights_only=True,
    save_best_only=True,
    monitor='val_accuracy',
    mode='max',
    verbose=1
)

In [8]:
history = model.fit(
    train_generator,
    epochs=15,
    batch_size=32,
    validation_data=validation_generator,
    verbose=1,
    callbacks=[checkpoint_callback, lr_reducer, early_stopping] # run again to see effects of lr_reducer and early_stopping
)

Epoch 1/15


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


251/251 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.6413 - loss: 0.7383
Epoch 1: val_accuracy improved from -inf to 0.72836, saving model to model_checkpoint.weights.h5
251/251 ━━━━━━━━━━━━━━━━━━━━ 1577s 6s/step - accuracy: 0.6415 - loss: 0.7379 - val_accuracy: 0.7284 - val_loss: 0.5647 - learning_rate: 1.0000e-05
Epoch 2/15
251/251 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7729 - loss: 0.5008
Epoch 2: val_accuracy improved from 0.72836 to 0.79240, saving model to model_checkpoint.weights.h5
251/251 ━━━━━━━━━━━━━━━━━━━━ 568s 2s/step - accuracy: 0.7730 - loss: 0.5006 - val_accuracy: 0.7924 - val_loss: 0.4593 - learning_rate: 1.0000e-05
Epoch 3/15
251/251 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.8018 - loss: 0.4356
Epoch 3: val_accuracy improved from 0.79240 to 0.84542, saving model to model_checkpoint.weights.h5
251/251 ━━━━━━━━━━━━━━━━━━━━ 561s 2s/step - accuracy: 0.8019 - loss: 0.4355 - val_accuracy: 0.8454 - val_loss: 0.3597 - learning_rate: 1.0000e-05
Epoch 4/15
251/251 ━

In [10]:
test_file_path = '/content/drive/MyDrive/xray_project/Test_PNG'
test_datagen = ImageDataGenerator(rescale=1.0/255.0)
test_generator = test_datagen.flow_from_directory(
    directory=test_file_path,
    target_size=(512, 512),
    color_mode='rgb',
    class_mode=None,
    batch_size=32,
    shuffle=False
)
predictions = model.predict(test_generator)

Found 2000 images belonging to 1 classes.
63/63 ━━━━━━━━━━━━━━━━━━━━ 63s 771ms/step


In [17]:
# could play around with this and see if it makes much difference in score
y_pred = np.where(predictions>0.5, 1, 0)

In [18]:
count0 = 0
count1 = 0
for i in range(len(y_pred)):
    if y_pred[i] == 0:
        count0 +=1
    else:
        count1 += 1
print(count0 / len(y_pred))
print(count1 / len(y_pred))

0.717
0.283


In [19]:
from datetime import datetime

y_submission = pd.read_csv('/content/drive/MyDrive/xray_project/data (4).csv')
y_submission = y_submission[-2000:].reset_index(drop=True)
y_submission.loc[0:1999, 'Finding'] = y_pred
y_submission['id'] = y_submission['id'].apply(lambda x: str(x).zfill(5))
y_submission.rename(columns={'Finding': 'Outcome'}, inplace=True)
y_submission['Outcome'] = y_submission['Outcome'].astype(int)
y_submission

y_submission.to_csv(f"submit_{datetime.today().strftime('%Y-%m-%d_%H%M%S')}.csv", index=False)

In [20]:
import zipfile

y_submission.to_csv('submission.csv', index=False)
with zipfile.ZipFile('submission.zip', 'w') as zipf:
    zipf.write('submission.csv')

array([[5.2111501e-01],
       [2.7750811e-02],
       [7.3657058e-02],
       [4.1312879e-04],
       [5.2581882e-01],
       [4.2738961e-03],
       [7.4825865e-01],
       [9.9385661e-01],
       [7.9838810e-03],
       [5.6037810e-03]], dtype=float32)